<a href="https://colab.research.google.com/github/aamuaamu/Bible_in_translations/blob/main/Bible_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy validate
#!pip uninstall ginza ja-ginza  # did not use it
#!pip install -U ginza ja-ginza-electra  # did not use it

!python -m spacy download en_core_web_sm
!python -m spacy download da_core_news_sm
!python -m spacy download nl_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download nb_core_news_sm
!python -m spacy download es_core_news_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download pt_core_news_sm
!python -m spacy download ro_core_news_sm
!python -m spacy download ru_core_news_sm
!python -m spacy download pl_core_news_sm
!python -m spacy download ja_core_news_sm
!python -m spacy download el_core_news_sm

In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import urllib.request
import spacy
import time
from google.colab import  drive
drive.mount('drive')

#start_time = time.time()

langs = ['English', 'Danish', 'Dutch', 'Norwegian', 'German', 'Portuguese', 'French', 
         'Italian', 'Romanian', 'Spanish', 'Russian', 'Polish', 'Japanese', 'Greek']
packages = ['en_core_web_sm', 'da_core_news_sm', 'nl_core_news_sm', 'nb_core_news_sm', 
            'de_core_news_sm','pt_core_news_sm','fr_core_news_sm','it_core_news_sm',
            'ro_core_news_sm','es_core_news_sm','ru_core_news_sm','pl_core_news_sm',
            'ja_core_news_sm','el_core_news_sm']
for lang, package in zip(langs, lackages):
    nlp = spacy.load(package)
    url_dir = 'https://raw.githubusercontent.com/christos-c/bible-corpus/master/bibles/'
    url_response = urllib.request.urlopen(url_dir + lang + ".xml")
    xml_content = ET.fromstring(url_response.read())
        
    wordDic = {}
    df = pd.DataFrame(columns = ["language","verseid", "book", "verseno", "versetext"])
    dg = pd.DataFrame()
    for n in xml_content.iter('seg'):
        s = n.text
        #if not s:  # For Italian and Dutch
        #    print(n.items()[0][1])
        #    continue
        #s = s.strip()
        #s = s.replace('--','') # For Spanish, Romanian
        #s = s.replace(' -', '') # For Romanian
        #s = s.replace('�', 'à') # For Portuguese
        #s = s.replace('``', '`') # For Romanian
        #s = s.replace(',,', ',') # For Romanian
        #s = s.replace("ß", 'ẞ') # For German
        while s.count('/ {*') > 0 and s.count('}') > 0:
            i = s.find('/ {*')
            j = s.find('}')
            s = s[:i] + s[j+1:]
        vid = n.items()[0][1]
        book = vid[2:5]
        verseno = vid[6:]
        df.loc[len(df.index)] = [lang, vid, book, verseno, s]
        if book not in wordDic:
            wordDic[book] = {}
            #print(book, (time.time() - start_time))
        for t in nlp(s):
            if t.pos_ not in wordDic[book]:
                wordDic[book][t.pos_] = {}
            w = t.lemma_.upper()
            if w in wordDic[book][t.pos_]:
                wordDic[book][t.pos_][w] += 1
            else:
                wordDic[book][t.pos_][w] = 1
    for book in wordDic:
        for p in wordDic[book]:
            row = pd.DataFrame({'language': [lang for w in wordDic[book][p]], 
                                'book':[book for w in wordDic[book][p]],
                                'pos': [p for w in wordDic[book][p]],
                                'word': [w for w in wordDic[book][p]], 
                                'count': [wordDic[book][p][w] for w in wordDic[book][p]]})
            dg = dg.append(row, ignore_index = True)

    dg.loc[dg['word'].isin(['`', '{', '}', '-']), 'pos'] = 'PUNCT'
    df.reset_index(inplace = True)
    df.to_csv('/content/drive/MyDrive/Colab Notebooks/Bible_' + lang + '.csv')
    dg.reset_index(inplace = True)
    dg.to_csv('/content/drive/MyDrive/Colab Notebooks/Counts_' + lang + '.csv')

#print("--- %s seconds ---" % (time.time() - start_time))